<a href="https://colab.research.google.com/github/callumselv/Y4_project/blob/main/Lohani_2021_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 16.5 MB/s eta 0:00:00


In [2]:
import numpy as np
from qutip import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from scipy.stats import unitary_group

In [3]:
no_data = 30000
d = 2
dimension = 2**d

In [4]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

In [5]:
cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

In [4]:
def gen_rhos_chos(no_data, d):
    dimension = 2**d
    rhos = np.zeros((no_data,dimension,dimension),dtype=complex)

    eta = 1e-7

    # Generate random density matrices with eta adjustment to avoid issues in Cholesky decomposition
    for i in range(no_data):
        random_unitary = unitary_group.rvs(dimension)
        coeffs = random_unitary[:,0]
        rho = np.outer(coeffs,coeffs.conj())
        rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

    # Calculate Cholesky decomposition of density matrices
    cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

    return rhos,cholesky_decompositions

In [8]:
for i in cholesky_decompositions[11000]:
    print(i)

[0.7367341+0.j 0.       +0.j 0.       +0.j 0.       +0.j]
[3.22038541e-01+0.36353001j 1.89377159e-04+0.j
 0.00000000e+00+0.j         0.00000000e+00+0.j        ]
[2.73129519e-01-1.21787911e-01j 1.06247803e-05-3.36880370e-05j
 1.66947350e-04+0.00000000e+00j 0.00000000e+00+0.00000000e+00j]
[ 1.19477978e-01-3.43005622e-01j -2.09691287e-05-3.74295840e-05j
  1.43099125e-05-1.52190244e-05j  1.69704006e-04+0.00000000e+00j]


In [9]:
all_taus = np.zeros((no_data, dimension**2))
for no in range(no_data):
    for cholesky in cholesky_decompositions:
        extracted_values = []
        for i in range(cholesky.shape[0]):
            extracted_values.append(cholesky[i, i].real)

        primary_indices = [(1, 0), (2, 1), (3, 2)]
        for i, j in primary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)

        secondary_indices = [(2, 0), (3, 1), (3, 0)]
        for i, j in secondary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)
    all_taus[no] = np.array(extracted_values)


KeyboardInterrupt: 

In [58]:
cholesky_decompositions.shape

(10000, 4, 4)

In [64]:
indices = np.zeros()

primary_indices = [(i, i - 1) for i in range(1, d**2)]
secondary_indices = [(i, i - 2) for i in range(2, d**2)]
secondary_indices = [(i, i - 2) for i in range(2, d**2)]
print(primary_indices, secondary_indices)


[(1, 0), (2, 1), (3, 2)] [(2, 0), (3, 1)]


In [10]:
def generate_index_pairs(d):
    d1 = 2**d - 1
    index_num = int(d1*(d1+1)/2)
    pairs = np.zeros((index_num,2), dtype=int)
    index=0
    for diff in range(1, dimension):
        for i in range(diff, dimension):
            j = i - diff
            pairs[index] = (i, j)
            index+=1

    return pairs

In [11]:
ind = generate_index_pairs(d)
ind

array([[1, 0],
       [2, 1],
       [3, 2],
       [2, 0],
       [3, 1],
       [3, 0]])

In [6]:
def gen_taus(no_data, d):
    rhos, cholesky_decompositions = gen_rhos_chos(no_data, d)

    dimension = 2**d
    tau_size = dimension**2

    all_taus = np.zeros((no_data, tau_size))
    indices = generate_index_pairs(d)

    for no in range(no_data):
        cholesky = cholesky_decompositions[no]
        extracted_values = np.zeros(tau_size)

        index=0
        for i in range(cholesky.shape[0]):
            extracted_values[i] = cholesky[i, i].real

        index = cholesky.shape[0]
        for i, j in indices:
            extracted_values[index] = cholesky[i, j].real
            extracted_values[index + 1] = cholesky[i, j].imag
            index += 2

        all_taus[no] = extracted_values

    return all_taus

In [8]:
def gen_nvals_taus(no_data, d):
    rhos, cholesky_decompositions = gen_rhos_chos(no_data, d)

    dimension = 2**d
    tau_size = dimension**2

    all_taus = np.zeros((no_data, tau_size))
    indices = generate_index_pairs(d)

    for no in range(no_data):
        cholesky = cholesky_decompositions[no]
        extracted_values = np.zeros(tau_size)

        index_tau=0
        for i in range(cholesky.shape[0]):
            extracted_values[i] = cholesky[i, i].real

        index_tau = cholesky.shape[0]
        for i, j in indices:
            extracted_values[index_tau] = cholesky[i, j].real
            extracted_values[index_tau + 1] = cholesky[i, j].imag
            index_tau += 2

        all_taus[no] = extracted_values

    nvals = np.zeros((no_data,6**2))

    for i1,rho in enumerate(rhos):
      for index,proj in enumerate(pauli_2d):
          nvals[i1,index] = np.trace(rho@proj)

    return nvals, all_taus

In [15]:
tau_train = gen_taus(no_data, d)

In [25]:
2**(2*d)

16

In [20]:
tau_train[np.random.randint(no_data)]

array([ 7.47908073e-02,  6.91677910e-04,  1.91193049e-04,  3.99655479e-04,
        2.62928813e-01, -1.79774284e-01, -7.71304588e-05,  4.51239282e-04,
        2.04081742e-04, -1.43593635e-04,  9.28119769e-02,  2.02140389e-01,
        8.11682687e-04,  1.70703853e-03,  7.93698664e-01,  4.62081262e-01])

In [13]:
tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values


In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values


In [14]:
# Make 6^d projectors of Pauli {X,Y,Z} operators

paulis = [sigmax(),sigmay(),sigmaz()]
evectors = np.zeros((6,2,1),dtype=complex)

i=0
for pauli in paulis:
    vals, vecs = pauli.eigenstates()
    evectors[i] = vecs[0][:]
    evectors[i+1] = vecs[1][:]
    i += 2

projectors = np.zeros((6,2,2), dtype=complex)

for i, vectors in enumerate(evectors):
    projectors[i] = np.outer(vectors,vectors.conj())

In [15]:
pauli_2d = np.zeros((6**2,4,4), dtype=complex)

index=0
for i in projectors:
    for j in projectors:
        pauli_2d[index] = np.kron(i,j)
        index += 1

In [13]:
np.kron(np.kron(projectors[0],projectors[0]),projectors[0]).shape

(8, 8)

In [11]:
pauli_3d = np.zeros((6**3,2**3,2**3), dtype=complex)

index=0
for i in pauli_2d:
    for j in projectors:
        pauli_3d[index] = np.kron(i,j)
        index += 1

In [16]:
nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

<ipython-input-16-09091a42e478>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [ ]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

primary_indices = [(1, 0), (2, 1), (3, 2)]
secondary_indices = [(2, 0), (3, 1), (3, 0)]

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values

nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

In [17]:
def reconstruct_matrix(taus):
    matrix = np.zeros((4, 4), dtype=np.complex_)

    index = 0
    for i in range(4):
        matrix[i, i] = taus[index]
        index += 1

    indices = generate_index_pairs(d)

    for i, j in indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    rho = matrix@(matrix.conj().T)
    rho = rho/np.trace(rho)

    return rho

In [33]:
# Make 6^d projectors of Pauli {X,Y,Z} operators

paulis = [sigmax(),sigmay(),sigmaz()]
evectors = np.zeros((6,2,1),dtype=complex)

i=0
for pauli in paulis:
    vals, vecs = pauli.eigenstates()
    evectors[i] = vecs[0][:]
    evectors[i+1] = vecs[1][:]
    i += 2

In [34]:
projectors = np.zeros((6,2,2), dtype=complex)

for i, vectors in enumerate(evectors):
    projectors[i] = np.outer(vectors,vectors.conj())

In [35]:
pauli_2d = np.zeros((6**2,4,4), dtype=complex)

index=0
for i in projectors:
    for j in projectors:
        pauli_2d[index] = np.kron(i,j)
        index += 1

In [46]:
((rho[0])@pauli_2d).shape

(36, 4)

In [47]:
nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

<ipython-input-47-09091a42e478>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [35]:
# Generate rho matrices
no_data = 30000

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

tau_size = dimension**2
all_taus = np.zeros((no_data, tau_size))

for no in range(no_data):
    cholesky = cholesky_decompositions[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    all_taus[no] = extracted_values

nvals = np.zeros((no_data,6**2))

for i1,rho in enumerate(rhos):
  for index,proj in enumerate(pauli_2d):
      nvals[i1,index] = np.trace(rho@proj)

n_min = np.min(nvals)
n_max = np.max(nvals)

nvals = (nvals - np.min(nvals)) / (np.max(nvals) - np.min(nvals))

<ipython-input-35-dc5d83edfd35>:37: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [61]:
np.abs(all_taus[np.random.randint(no_data)]).min()

1.4504845007285327e-05

In [92]:
shift = np.abs(np.min(all_taus)) + 1  # Shift so all values are positive
taus_shifted = all_taus + shift
taus_shifted_log = np.log(taus_shifted)

In [95]:
taus_shifted_log[np.random.randint(no_data)].min()

0.35889741288145555

In [73]:
def unlog_unshift(taus, shift):
    taus_unlog = np.exp(taus) - shift
    return taus_unlog

In [118]:
def build_cnn_model():
    model = Sequential()

    model.add(tf.keras.layers.Reshape((6, 6, 1), input_shape=(36,)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))

    model.add(Flatten())

    model.add(Dense(750, activation='relu'))


    model.add(Dense(450, activation='relu'))


    model.add(Dense(2**(2*d), activation='linear'))

    return model

model = build_cnn_model()

model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.005),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape_3 (Reshape)                  │ (None, 6, 6, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 5, 5, 25)            │             125 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 2, 2, 25)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 1, 1, 25)            │           2,525 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 750)                 │          19,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 450)                 │         337,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 16)                  │           7,216 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 367,316 (1.40 MB)

 Trainable params: 367,316 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
nvals, all_taus = gen_nvals_taus(30000,2)

<ipython-input-8-b5b2d884fa7e>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[i1,index] = np.trace(rho@proj)


In [19]:
all_taus.shape

(30000, 16)

In [20]:
nvals.shape

(30000, 36)

In [119]:
# `x_train`, `y_train` are probabilities (n values) and labels (tau-vectors)
model.fit(nvals, all_taus, epochs=300, batch_size=100, validation_split=0.2)

Epoch 1/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0585 - mean_absolute_error: 0.1355 - val_loss: 0.0520 - val_mean_absolute_error: 0.1273
Epoch 2/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0513 - mean_absolute_error: 0.1258 - val_loss: 0.0498 - val_mean_absolute_error: 0.1230
Epoch 3/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0498 - mean_absolute_error: 0.1227 - val_loss: 0.0492 - val_mean_absolute_error: 0.1213
Epoch 4/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0493 - mean_absolute_error: 0.1212 - val_loss: 0.0491 - val_mean_absolute_error: 0.1206
Epoch 5/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0492 - mean_absolute_error: 0.1208 - val_loss: 0.0490 - val_mean_absolute_error: 0.1204
Epoch 6/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0492 - mean_absolute_error: 0.1207 - val_loss: 0.0489 - val_mean_absolute_error: 0.1203
Epoch 7/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0490 - mean_absolute_error: 0.1

In [134]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_cnn_model()
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mean_squared_error']
)

# Add early stopping callback to prevent overtraining
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    nvals, all_taus,
    epochs=300,
    batch_size=100,
    validation_split=0.2,
    callbacks=[early_stopping]
)


Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.1162 - mean_squared_error: 0.0459 - val_loss: 0.0896 - val_mean_squared_error: 0.0308
Epoch 2/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0861 - mean_squared_error: 0.0290 - val_loss: 0.0755 - val_mean_squared_error: 0.0239
Epoch 3/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0738 - mean_squared_error: 0.0232 - val_loss: 0.0672 - val_mean_squared_error: 0.0200
Epoch 4/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0662 - mean_squared_error: 0.0194 - val_loss: 0.0597 - val_mean_squared_error: 0.0161
Epoch 5/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0594 - mean_squared_error: 0.0161 - val_loss: 0.0549 - val_mean_squared_error: 0.0138
Epoch 6/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0548 - mean_squared_error: 0.0140 - val_loss: 0.0513 - val_mean_squared_error: 0.0122
Epoch 7/300
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0507 - mean_squared_error: 0.0123 - val_loss: 0.0482 - 

In [150]:
# Generate rho matrices
no_test = 1

test_rhos = np.zeros((no_test,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_test):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    test_rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

cholesky_decompositions_t = np.array([np.linalg.cholesky(rho) for rho in test_rhos])

tau_size = dimension**2
test_taus = np.zeros((no_test, tau_size))

for no in range(no_test):
    cholesky = cholesky_decompositions_t[no]
    extracted_values = np.zeros(tau_size)

    for i in range(cholesky.shape[0]):
        extracted_values[i] = cholesky[i, i].real

    index = cholesky.shape[0]
    for i, j in ind:
        extracted_values[index] = cholesky[i, j].real
        extracted_values[index + 1] = cholesky[i, j].imag
        index += 2

    test_taus[no] = extracted_values

nvals_t = np.zeros((no_test,6**2))

for i1,rho in enumerate(test_rhos):
  for index,proj in enumerate(pauli_2d):
      nvals_t[i1,index] = np.trace(rho@proj)

shift_t = np.abs(np.min(test_taus)) + 1  # Shift so all values are positive
taus_t_shifted = test_taus + 2
taus_t_shifted_log = np.log(taus_t_shifted)

<ipython-input-150-230aa0df8e96>:37: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals_t[i1,index] = np.trace(rho@proj)


In [149]:
print(np.min(nvals))
print(np.max(nvals))

0.0
1.0


In [151]:
tau_nn = model.predict(nvals_t)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [152]:
for i in range(len(tau_nn[0])):
    print(tau_nn[0,i],test_taus[0,i])

0.5371624 0.5257821498076084
0.0007096551 0.00016558421698149864
0.00016883017 0.000249638919107115
-0.0007570129 0.00018187496672064188
0.041995242 0.06895731200428411
-0.16272397 -0.14827102759802435
1.6191218e-05 -5.054292815108513e-05
0.0004988355 -3.2483250377041935e-05
-1.0312226e-05 8.596593542801998e-05
-0.0002111858 -6.833238988607688e-05
-0.57886195 -0.5684628810028554
0.38354665 0.35977967842484815
-0.0012810384 -4.391223745567451e-05
0.00030996412 4.423353698029809e-06
-0.15800397 -0.16243991687706466
0.47313604 0.466728391386248


In [153]:
mse = 0
for i in range(len(test_taus[0])):
    mse += (test_taus[0,i]-tau_nn[0,i])**2

print(mse)

0.0018022209660357729


In [145]:
test_cho = reconstruct_matrix(test_taus[0])

In [146]:
test_rho = test_cho@test_cho.conj().T

In [147]:
test_rho

array([[ 0.2783737 +0.j        ,  0.03995763-0.03589901j,
        -0.26144822-0.26876138j,  0.2199188 -0.09510134j],
       [ 0.03995763+0.03589901j,  0.01036503+0.j        ,
        -0.00286875-0.07229418j,  0.04383128+0.01470988j],
       [-0.26144822+0.26876138j, -0.00286875+0.07229418j,
         0.5050328 +0.j        , -0.11472999+0.30164399j],
       [ 0.2199188 +0.09510134j,  0.04383128-0.01470988j,
        -0.11472999-0.30164399j,  0.20622832+0.j        ]])

In [148]:
test_rhos[0]

array([[ 0.27837374-6.91284099e-18j,  0.03995763-3.58990095e-02j,
        -0.26144823-2.68761391e-01j,  0.21991881-9.51013421e-02j],
       [ 0.03995763+3.58990095e-02j,  0.01036505+2.05126953e-19j,
        -0.00286875-7.22941833e-02j,  0.04383129+1.47098776e-02j],
       [-0.26144823+2.68761391e-01j, -0.00286875+7.22941833e-02j,
         0.50503285+8.94248169e-18j, -0.11472999+3.01644006e-01j],
       [ 0.21991881+9.51013421e-02j,  0.04383129-1.47098776e-02j,
        -0.11472999-3.01644006e-01j,  0.20622836+4.08533396e-20j]])

In [155]:
rho_nn = reconstruct_matrix(tau_nn[0])

In [158]:
rho_nn

array([[ 0.27537991+0.j        ,  0.02152914+0.08342153j,
        -0.29675745-0.19662776j, -0.08100179-0.24255635j],
       [ 0.02152914-0.08342153j,  0.02695472+0.j        ,
        -0.08276537+0.07452483j, -0.07981178+0.00557486j],
       [-0.29675745+0.19662776j, -0.08276537-0.07452483j,
         0.46019164+0.j        ,  0.26048099+0.20354805j],
       [-0.08100179+0.24255635j, -0.07981178-0.00557486j,
         0.26048099-0.20354805j,  0.23747373+0.j        ]])

In [159]:
test_rhos[0]

array([[ 0.27644687+6.04728294e-18j,  0.03625652+7.79582596e-02j,
        -0.29888764-1.89165733e-01j, -0.08540801-2.45397457e-01j],
       [ 0.03625652-7.79582596e-02j,  0.02673944-3.84512384e-19j,
        -0.09254458+5.94771414e-02j, -0.08040373-8.09920264e-03j],
       [-0.29888764+1.89165733e-01j, -0.09254458-5.94771414e-02j,
         0.45259153+2.07558995e-19j,  0.26026048+2.06875204e-01j],
       [-0.08540801+2.45397457e-01j, -0.08040373+8.09920264e-03j,
         0.26026048-2.06875204e-01j,  0.24422217+8.01907437e-19j]])

In [156]:
from scipy.linalg import sqrtm

In [160]:
def fidelity(rho_t, rho_nn):
    sqrt_nn = sqrtm(rho_nn)
    matrix = sqrt_nn@rho_t@sqrt_nn
    tr = np.trace(sqrtm(matrix))
    return np.abs(tr)**2

In [161]:
fidelity(test_rhos[0], rho_nn)

0.99948127099706151087